In [6]:
import numpy as np
import skfuzzy as fuzz
# !from skfuzzy import control as ctrl
from skfuzzy import control as ctrl

In [8]:


class NPKComplexFertilizerCalculator_sugarcane:
    def __init__(self, crop_name, soil_n, soil_p, soil_k):
        self.crop_name = crop_name  # Name of the crop
        self.soil_n = soil_n  # Current Nitrogen level (kg/ha)
        self.soil_p = soil_p  # Current Phosphorus level (kg/ha)
        self.soil_k = soil_k  # Current Potassium level (kg/ha)

        # Updated Recommended NPK needs based on Suru sugarcane
        self.crop_n_needs = 170  # Recommended Nitrogen (kg/ha)
        self.crop_p_needs = 85   # Recommended Phosphorus (kg/ha)
        self.crop_k_needs = 85   # Recommended Potassium (kg/ha)

        # Initialize fuzzy variables
        self._initialize_fuzzy_system()

        # Updated growth stages for Suru sugarcane
        self.growth_stages = {
            "Germination": {"days": (0, 40), "NPK": (20, 10, 10)},
            "Tillering": {"days": (41, 135), "NPK": (30, 20, 20)},
            "Grand Growth": {"days": (136, 300), "NPK": (40, 35, 35)},
            "Maturity": {"days": (301, 360), "NPK": (20, 15, 15)},
            "Harvesting": {"days": (360, 400), "NPK": (10, 5, 5)}
        }

        # NPK complex fertilizers and individual fertilizers
        self.fertilizers = {
            "10:26:26": {"N": 0.10, "P": 0.26, "K": 0.26},
            "12:32:16": {"N": 0.12, "P": 0.32, "K": 0.16},
            "15:15:15": {"N": 0.15, "P": 0.15, "K": 0.15},
            "Urea (46-0-0)": {"N": 0.46, "P": 0.0, "K": 0.0},
            "DAP (18-46-0)": {"N": 0.18, "P": 0.46, "K": 0.0},
            "MOP (0-0-60)": {"N": 0.0, "P": 0.0, "K": 0.60},
            "Ammonium Sulfate (21-0-0)": {"N": 0.21, "P": 0.0, "K": 0.0},
            "CAN (26-0-0)": {"N": 0.26, "P": 0.0, "K": 0.0},
            "SSP (0-16-0)": {"N": 0.0, "P": 0.16, "K": 0.0},
            "TSP (0-44-0)": {"N": 0.0, "P": 0.44, "K": 0.0},
            "SOP (0-0-50)": {"N": 0.0, "P": 0.0, "K": 0.50},
        }

    def _initialize_fuzzy_system(self):
        # Define fuzzy variables
        self.n_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'n_needed')  # Expanded range
        self.p_needed = ctrl.Antecedent(np.arange(0, 101, 1), 'p_needed')  # Expanded range
        self.k_needed = ctrl.Antecedent(np.arange(0, 101, 1), 'k_needed')

        self.n_fertilizer = ctrl.Consequent(np.arange(0, 131, 1), 'n_fertilizer')
        self.p_fertilizer = ctrl.Consequent(np.arange(0, 101, 1), 'p_fertilizer')
        self.k_fertilizer = ctrl.Consequent(np.arange(0, 51, 1), 'k_fertilizer')

        # Membership functions for inputs (N, P, K needs)
        self.n_needed['low'] = fuzz.trimf(self.n_needed.universe, [0, 0, 100])
        self.n_needed['medium'] = fuzz.trimf(self.n_needed.universe, [50, 100, 150])
        self.n_needed['high'] = fuzz.trimf(self.n_needed.universe, [100, 200, 200])

        self.p_needed['low'] = fuzz.trimf(self.p_needed.universe, [0, 0, 50])
        self.p_needed['medium'] = fuzz.trimf(self.p_needed.universe, [25, 50, 75])
        self.p_needed['high'] = fuzz.trimf(self.p_needed.universe, [50, 100, 100])

        self.k_needed['low'] = fuzz.trimf(self.k_needed.universe, [0, 0, 50])
        self.k_needed['medium'] = fuzz.trimf(self.k_needed.universe, [25, 40, 50])
        self.k_needed['high'] = fuzz.trimf(self.k_needed.universe, [40, 50, 50])

        # Define membership functions for outputs
        self.n_fertilizer['low'] = fuzz.trimf(self.n_fertilizer.universe, [0, 0, 65])
        self.n_fertilizer['medium'] = fuzz.trimf(self.n_fertilizer.universe, [30, 65, 100])
        self.n_fertilizer['high'] = fuzz.trimf(self.n_fertilizer.universe, [65, 130, 130])

        self.p_fertilizer['low'] = fuzz.trimf(self.p_fertilizer.universe, [0, 0, 30])
        self.p_fertilizer['medium'] = fuzz.trimf(self.p_fertilizer.universe, [15, 30, 60])
        self.p_fertilizer['high'] = fuzz.trimf(self.p_fertilizer.universe, [30, 100, 100])

        self.k_fertilizer['low'] = fuzz.trimf(self.k_fertilizer.universe, [0, 0, 25])
        self.k_fertilizer['medium'] = fuzz.trimf(self.k_fertilizer.universe, [10, 25, 30])
        self.k_fertilizer['high'] = fuzz.trimf(self.k_fertilizer.universe, [25, 50, 50])

        # Define fuzzy rules
       # Define fuzzy rules
        # Define fuzzy rules
self.rules = [
    ctrl.Rule(self.n_needed['low'] & self.p_needed['low'] & self.k_needed['low'],
              (self.n_fertilizer['low'], self.p_fertilizer['low'], self.k_fertilizer['low'])),
    ctrl.Rule(self.n_needed['medium'] & self.p_needed['medium'] & self.k_needed['medium'],
              (self.n_fertilizer['medium'], self.p_fertilizer['medium'], self.k_fertilizer['medium'])),
    ctrl.Rule(self.n_needed['high'] & self.p_needed['high'] & self.k_needed['high'],
              (self.n_fertilizer['high'], self.p_fertilizer['high'], self.k_fertilizer['high'])),

    # Catch-all rule for when inputs don't match any specific rule
    ctrl.Rule(self.n_needed['low'] | self.n_needed['medium'] | self.n_needed['high'],
              self.n_fertilizer['medium']),
    ctrl.Rule(self.p_needed['low'] | self.p_needed['medium'] | self.p_needed['high'],
              self.p_fertilizer['medium']),
    ctrl.Rule(self.k_needed['low'] | self.k_needed['medium'] | self.k_needed['high'],
              self.k_fertilizer['medium']),
]



        # Create control system
        self.fertilizer_ctrl = ctrl.ControlSystem(self.rules)
        self.fertilizer_sim = ctrl.ControlSystemSimulation(self.fertilizer_ctrl)

    def fuzzy_logic(self, growth_stage):
        # Get NPK needs based on growth stage
        n_needed_target, p_needed_target, k_needed_target = self.get_npk_needs(growth_stage)

        # Fuzzy logic to calculate fertilizer required
        n_needed = max(0, n_needed_target - self.soil_n)
        p_needed = max(0, p_needed_target - self.soil_p)
        k_needed = max(0, k_needed_target - self.soil_k)

        # Set inputs to the fuzzy logic system
        self.fertilizer_sim.input['n_needed'] = n_needed
        self.fertilizer_sim.input['p_needed'] = p_needed
        self.fertilizer_sim.input['k_needed'] = k_needed

        # Compute the fuzzy logic output
        self.fertilizer_sim.compute()

        # Get the fertilizer amounts needed
        n_fertilizer_needed = self.fertilizer_sim.output['n_fertilizer']
        p_fertilizer_needed = self.fertilizer_sim.output['p_fertilizer']
        k_fertilizer_needed = self.fertilizer_sim.output['k_fertilizer']

        return n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed

    def get_npk_needs(self, growth_stage):
        # Get the NPK needs based on the growth stage
        return self.growth_stages[growth_stage]['NPK']

    def calculate_fertilizer_amounts(self, n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed):
        # Calculate the amount of each fertilizer needed while considering interactions
        amounts = {}
        total_n_applied = total_p_applied = total_k_applied = 0

        for fertilizer, nutrients in self.fertilizers.items():
            n_content = nutrients["N"]
            p_content = nutrients["P"]
            k_content = nutrients["K"]

            # Adjust remaining needs by subtracting already applied nutrients
            n_needed_remaining = max(0, n_fertilizer_needed - total_n_applied)
            p_needed_remaining = max(0, p_fertilizer_needed - total_p_applied)
            k_needed_remaining = max(0, k_fertilizer_needed - total_k_applied)

            # Calculate how much of the current fertilizer is required
            n_amount = n_needed_remaining / n_content if n_content > 0 else 0
            p_amount = p_needed_remaining / p_content if p_content > 0 else 0
            k_amount = k_needed_remaining / k_content if k_content > 0 else 0

            # Use the maximum amount required based on N, P, or K content
            amount_to_apply = max(n_amount, p_amount, k_amount)
            amounts[fertilizer] = amount_to_apply

            # Update the total applied nutrients
            total_n_applied += amount_to_apply * n_content
            total_p_applied += amount_to_apply * p_content
            total_k_applied += amount_to_apply * k_content

        return amounts

    def display_fertilizer_plan(self):
        # Initialize a list to store fertilizer plans for each growth stage
        fertilizer_plan_list = []
    
        print(f"\nFertilizer plan for {self.crop_name}:")
        for phase, stage_data in self.growth_stages.items():
            # Get fertilizer amounts using fuzzy logic
            n_fertilizer, p_fertilizer, k_fertilizer = self.fuzzy_logic(phase)
            fertilizer_amounts = self.calculate_fertilizer_amounts(n_fertilizer, p_fertilizer, k_fertilizer)
    
            # Display the data
            print(f"{phase} (Days: {stage_data['days'][0]} to {stage_data['days'][1]}):")
            print(f"  N fertilizer needed: {n_fertilizer:.2f} kg/ha")
            print(f"  P fertilizer needed: {p_fertilizer:.2f} kg/ha")
            print(f"  K fertilizer needed: {k_fertilizer:.2f} kg/ha")
    
            for fertilizer, amount in fertilizer_amounts.items():
                print(f"    {fertilizer}: {amount:.2f} kg/ha")
    
            print()
    
            # Store the data for this phase in a dictionary
            phase_data = {
                'phase': phase,
                'days': stage_data['days'],
                'n_fertilizer_needed': n_fertilizer,
                'p_fertilizer_needed': p_fertilizer,
                'k_fertilizer_needed': k_fertilizer,
                'specific_fertilizer_amounts': fertilizer_amounts  # Dictionary with the specific fertilizer amounts
            }
    
            # Append the dictionary to the list
            fertilizer_plan_list.append(phase_data)
    
        # Return the list containing all phases' data
        return fertilizer_plan_list
      # Blank line for readability

# Example usage
calculator = NPKComplexFertilizerCalculator_sugarcane(crop_name="Sugarcane", soil_n=50, soil_p=20, soil_k=30)
calculator.display_fertilizer_plan()




Fertilizer plan for Sugarcane:
Germination (Days: 0 to 40):
  N fertilizer needed: 21.67 kg/ha
  P fertilizer needed: 10.00 kg/ha
  K fertilizer needed: 8.33 kg/ha
    10:26:26: 216.67 kg/ha
    12:32:16: 0.00 kg/ha
    15:15:15: 0.00 kg/ha
    Urea (46-0-0): 0.00 kg/ha
    DAP (18-46-0): 0.00 kg/ha
    MOP (0-0-60): 0.00 kg/ha
    Ammonium Sulfate (21-0-0): 0.00 kg/ha
    CAN (26-0-0): 0.00 kg/ha
    SSP (0-16-0): 0.00 kg/ha
    TSP (0-44-0): 0.00 kg/ha
    SOP (0-0-50): 0.00 kg/ha

Tillering (Days: 41 to 135):
  N fertilizer needed: 21.67 kg/ha
  P fertilizer needed: 10.00 kg/ha
  K fertilizer needed: 8.33 kg/ha
    10:26:26: 216.67 kg/ha
    12:32:16: 0.00 kg/ha
    15:15:15: 0.00 kg/ha
    Urea (46-0-0): 0.00 kg/ha
    DAP (18-46-0): 0.00 kg/ha
    MOP (0-0-60): 0.00 kg/ha
    Ammonium Sulfate (21-0-0): 0.00 kg/ha
    CAN (26-0-0): 0.00 kg/ha
    SSP (0-16-0): 0.00 kg/ha
    TSP (0-44-0): 0.00 kg/ha
    SOP (0-0-50): 0.00 kg/ha

Grand Growth (Days: 136 to 300):
  N fertilizer need

[{'phase': 'Germination',
  'days': (0, 40),
  'n_fertilizer_needed': 21.666666666666664,
  'p_fertilizer_needed': 9.999999999999998,
  'k_fertilizer_needed': 8.333333333333332,
  'specific_fertilizer_amounts': {'10:26:26': 216.66666666666663,
   '12:32:16': 0.0,
   '15:15:15': 0.0,
   'Urea (46-0-0)': 0.0,
   'DAP (18-46-0)': 0.0,
   'MOP (0-0-60)': 0,
   'Ammonium Sulfate (21-0-0)': 0.0,
   'CAN (26-0-0)': 0.0,
   'SSP (0-16-0)': 0,
   'TSP (0-44-0)': 0,
   'SOP (0-0-50)': 0}},
 {'phase': 'Tillering',
  'days': (41, 135),
  'n_fertilizer_needed': 21.666666666666664,
  'p_fertilizer_needed': 9.999999999999998,
  'k_fertilizer_needed': 8.333333333333332,
  'specific_fertilizer_amounts': {'10:26:26': 216.66666666666663,
   '12:32:16': 0.0,
   '15:15:15': 0.0,
   'Urea (46-0-0)': 0.0,
   'DAP (18-46-0)': 0.0,
   'MOP (0-0-60)': 0,
   'Ammonium Sulfate (21-0-0)': 0.0,
   'CAN (26-0-0)': 0.0,
   'SSP (0-16-0)': 0,
   'TSP (0-44-0)': 0,
   'SOP (0-0-50)': 0}},
 {'phase': 'Grand Growth',
  '

In [20]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class NPKComplexFertilizerCalculator_sugarcane:
    def __init__(self, crop_name, soil_n, soil_p, soil_k):
        self.crop_name = crop_name  # Name of the crop
        self.soil_n = soil_n  # Current Nitrogen level (kg/ha)
        self.soil_p = soil_p  # Current Phosphorus level (kg/ha)
        self.soil_k = soil_k  # Current Potassium level (kg/ha)

        # Updated Recommended NPK needs based on Suru sugarcane
        self.crop_n_needs = 170  # Recommended Nitrogen (kg/ha)
        self.crop_p_needs = 85   # Recommended Phosphorus (kg/ha)
        self.crop_k_needs = 85   # Recommended Potassium (kg/ha)

        # Initialize fuzzy variables
        self._initialize_fuzzy_system()

        # Updated growth stages for Suru sugarcane with water deficit adjustments
        self.growth_stages = {
            "Germination": {"days": (0, 40), "NPK": (20, 10, 10), "water_deficit": 0.0},
            "Tillering": {"days": (41, 135), "NPK": (50, 25, 25), "water_deficit": 0.0},  # 100% irrigation
            "Grand Growth": {"days": (136, 300), "NPK": (50, 35, 35), "water_deficit": 0.30},  # 30% water deficit
            "Maturity": {"days": (301, 360), "NPK": (20, 15, 15), "water_deficit": 0.60},  # 60% water deficit
            "Harvesting": {"days": (360, 400), "NPK": (10, 5, 5), "water_deficit": 0.60}   # Maintain water deficit
        }

        # NPK complex fertilizers and individual fertilizers
        self.fertilizers = {
            "10:26:26": {"N": 0.10, "P": 0.26, "K": 0.26},
            "12:32:16": {"N": 0.12, "P": 0.32, "K": 0.16},
            "15:15:15": {"N": 0.15, "P": 0.15, "K": 0.15},
            "Urea (46-0-0)": {"N": 0.46, "P": 0.0, "K": 0.0},
            "DAP (18-46-0)": {"N": 0.18, "P": 0.46, "K": 0.0},
            "MOP (0-0-60)": {"N": 0.0, "P": 0.0, "K": 0.60},
            "Ammonium Sulfate (21-0-0)": {"N": 0.21, "P": 0.0, "K": 0.0},
            "CAN (26-0-0)": {"N": 0.26, "P": 0.0, "K": 0.0},
            "SSP (0-16-0)": {"N": 0.0, "P": 0.16, "K": 0.0},
            "TSP (0-44-0)": {"N": 0.0, "P": 0.44, "K": 0.0},
            "SOP (0-0-50)": {"N": 0.0, "P": 0.0, "K": 0.50},
        }

    def _initialize_fuzzy_system(self):
        # Define fuzzy variables
        self.n_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'n_needed')
        self.p_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'p_needed')
        self.k_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'k_needed')

        self.n_fertilizer = ctrl.Consequent(np.arange(0, 131, 1), 'n_fertilizer')
        self.p_fertilizer = ctrl.Consequent(np.arange(0, 101, 1), 'p_fertilizer')
        self.k_fertilizer = ctrl.Consequent(np.arange(0, 51, 1), 'k_fertilizer')

        # Membership functions for inputs (N, P, K needs)
        self.n_needed['low'] = fuzz.trimf(self.n_needed.universe, [0, 0, 100])
        self.n_needed['medium'] = fuzz.trimf(self.n_needed.universe, [50, 100, 150])
        self.n_needed['high'] = fuzz.trimf(self.n_needed.universe, [100, 200, 200])

        self.p_needed['low'] = fuzz.trimf(self.p_needed.universe, [0, 0, 50])
        self.p_needed['medium'] = fuzz.trimf(self.p_needed.universe, [25, 50, 75])
        self.p_needed['high'] = fuzz.trimf(self.p_needed.universe, [50, 100, 100])

        self.k_needed['low'] = fuzz.trimf(self.k_needed.universe, [0, 0, 50])
        self.k_needed['medium'] = fuzz.trimf(self.k_needed.universe, [25, 40, 50])
        self.k_needed['high'] = fuzz.trimf(self.k_needed.universe, [40, 50, 50])

        # Define membership functions for outputs
        self.n_fertilizer['low'] = fuzz.trimf(self.n_fertilizer.universe, [0, 0, 65])
        self.n_fertilizer['medium'] = fuzz.trimf(self.n_fertilizer.universe, [30, 65, 100])
        self.n_fertilizer['high'] = fuzz.trimf(self.n_fertilizer.universe, [65, 130, 130])

        self.p_fertilizer['low'] = fuzz.trimf(self.p_fertilizer.universe, [0, 0, 30])
        self.p_fertilizer['medium'] = fuzz.trimf(self.p_fertilizer.universe, [15, 30, 60])
        self.p_fertilizer['high'] = fuzz.trimf(self.p_fertilizer.universe, [30, 100, 100])

        self.k_fertilizer['low'] = fuzz.trimf(self.k_fertilizer.universe, [0, 0, 25])
        self.k_fertilizer['medium'] = fuzz.trimf(self.k_fertilizer.universe, [10, 25, 30])
        self.k_fertilizer['high'] = fuzz.trimf(self.k_fertilizer.universe, [25, 50, 50])

        # Define fuzzy rules
        self.rules = [
            ctrl.Rule(self.n_needed['low'] & self.p_needed['low'] & self.k_needed['low'],
                      (self.n_fertilizer['low'], self.p_fertilizer['low'], self.k_fertilizer['low'])),
            ctrl.Rule(self.n_needed['medium'] & self.p_needed['medium'] & self.k_needed['medium'],
                      (self.n_fertilizer['medium'], self.p_fertilizer['medium'], self.k_fertilizer['medium'])),
            ctrl.Rule(self.n_needed['high'] & self.p_needed['high'] & self.k_needed['high'],
                      (self.n_fertilizer['high'], self.p_fertilizer['high'], self.k_fertilizer['high'])),
        ]

        # Create control system
        self.fertilizer_ctrl = ctrl.ControlSystem(self.rules)
        self.fertilizer_sim = ctrl.ControlSystemSimulation(self.fertilizer_ctrl)

    def fuzzy_logic(self, growth_stage):
        # Get NPK needs based on growth stage
        n_needed_target, p_needed_target, k_needed_target = self.get_npk_needs(growth_stage)

        # Fuzzy logic to calculate fertilizer required
        n_needed = max(0, n_needed_target - self.soil_n)
        p_needed = max(0, p_needed_target - self.soil_p)
        k_needed = max(0, k_needed_target - self.soil_k)

        # Set inputs to the fuzzy logic system
        self.fertilizer_sim.input['n_needed'] = n_needed
        self.fertilizer_sim.input['p_needed'] = p_needed
        self.fertilizer_sim.input['k_needed'] = k_needed

        # Compute the fuzzy logic output
        self.fertilizer_sim.compute()

        # Get the fertilizer amounts needed
        n_fertilizer_needed = self.fertilizer_sim.output['n_fertilizer']
        p_fertilizer_needed = self.fertilizer_sim.output['p_fertilizer']
        k_fertilizer_needed = self.fertilizer_sim.output['k_fertilizer']

        return n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed

    def get_npk_needs(self, growth_stage):
        # Get the NPK needs based on the growth stage
        return self.growth_stages[growth_stage]['NPK']
    def calculate_fertilizer_amounts(self, n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed):
            amounts = {}
            total_n_applied = total_p_applied = total_k_applied = 0
        
            # Apply nitrogen first with high-N fertilizers
            for fertilizer, nutrients in self.fertilizers.items():
                if nutrients["N"] > 0:
                    n_content = nutrients["N"]
                    n_needed_remaining = max(0, n_fertilizer_needed - total_n_applied)
                    n_amount = n_needed_remaining / n_content if n_content > 0 else 0
                    amounts[fertilizer] = n_amount
                    total_n_applied += n_amount * n_content
        
            # Apply phosphorus with phosphate-heavy fertilizers
            for fertilizer, nutrients in self.fertilizers.items():
                if nutrients["P"] > 0:
                    p_content = nutrients["P"]
                    p_needed_remaining = max(0, p_fertilizer_needed - total_p_applied)
                    p_amount = p_needed_remaining / p_content if p_content > 0 else 0
                    amounts[fertilizer] = max(amounts.get(fertilizer, 0), p_amount)
                    total_p_applied += p_amount * p_content
        
            # Apply potassium with potassium-heavy fertilizers
            for fertilizer, nutrients in self.fertilizers.items():
                if nutrients["K"] > 0:
                    k_content = nutrients["K"]
                    k_needed_remaining = max(0, k_fertilizer_needed - total_k_applied)
                    k_amount = k_needed_remaining / k_content if k_content > 0 else 0
                    amounts[fertilizer] = max(amounts.get(fertilizer, 0), k_amount)
                    total_k_applied += k_amount * k_content
        
            return amounts



    def display_fertilizer_plan(self):
        # Initialize a list to store fertilizer plans for each growth stage
        fertilizer_plan_list = []

        print(f"\nFertilizer plan for {self.crop_name}:")
        for phase, stage_data in self.growth_stages.items():
            # Get fertilizer amounts using fuzzy logic
            n_fertilizer, p_fertilizer, k_fertilizer = self.fuzzy_logic(phase)
            fertilizer_amounts = self.calculate_fertilizer_amounts(n_fertilizer, p_fertilizer, k_fertilizer)

            # Display the data
            print(f"{phase} (Days: {stage_data['days'][0]} to {stage_data['days'][1]}):")
            print(f"  N fertilizer needed: {n_fertilizer:.2f} kg/ha")
            print(f"  P fertilizer needed: {p_fertilizer:.2f} kg/ha")
            print(f"  K fertilizer needed: {k_fertilizer:.2f} kg/ha")

            for fertilizer, amount in fertilizer_amounts.items():
                print(f"    {fertilizer}: {amount:.2f} kg/ha")

            print()

            # Store the data for this phase in a dictionary
            phase_data = {
                'phase': phase,
                'days': stage_data['days'],
                'n_fertilizer_needed': n_fertilizer,
                'p_fertilizer_needed': p_fertilizer,
                'k_fertilizer_needed': k_fertilizer,
                'specific_fertilizer_amounts': fertilizer_amounts  # Dictionary with the specific fertilizer amounts
            }

            # Append the dictionary to the list
            fertilizer_plan_list.append(phase_data)

        # Return the list containing all phases' data
        return fertilizer_plan_list


# Example usage
calculator = NPKComplexFertilizerCalculator_sugarcane(crop_name="Sugarcane", soil_n=600, soil_p=20, soil_k=-20)
calculator.display_fertilizer_plan()



Fertilizer plan for Sugarcane:
Germination (Days: 0 to 40):
  N fertilizer needed: 26.54 kg/ha
  P fertilizer needed: 12.25 kg/ha
  K fertilizer needed: 10.21 kg/ha
    10:26:26: 265.42 kg/ha
    12:32:16: 0.00 kg/ha
    15:15:15: 0.00 kg/ha
    Urea (46-0-0): 0.00 kg/ha
    DAP (18-46-0): 0.00 kg/ha
    Ammonium Sulfate (21-0-0): 0.00 kg/ha
    CAN (26-0-0): 0.00 kg/ha
    SSP (0-16-0): 0.00 kg/ha
    TSP (0-44-0): 0.00 kg/ha
    MOP (0-0-60): 0.00 kg/ha
    SOP (0-0-50): 0.00 kg/ha

Tillering (Days: 41 to 135):
  N fertilizer needed: 30.90 kg/ha
  P fertilizer needed: 14.26 kg/ha
  K fertilizer needed: 11.89 kg/ha
    10:26:26: 309.04 kg/ha
    12:32:16: 0.00 kg/ha
    15:15:15: 0.00 kg/ha
    Urea (46-0-0): 0.00 kg/ha
    DAP (18-46-0): 0.00 kg/ha
    Ammonium Sulfate (21-0-0): 0.00 kg/ha
    CAN (26-0-0): 0.00 kg/ha
    SSP (0-16-0): 0.00 kg/ha
    TSP (0-44-0): 0.00 kg/ha
    MOP (0-0-60): 0.00 kg/ha
    SOP (0-0-50): 0.00 kg/ha



KeyError: 'n_fertilizer'

In [5]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class NPKComplexFertilizerCalculator_sugarcane:
    def __init__(self, crop_name, soil_n, soil_p, soil_k, biofertilizer=False):
        self.crop_name = crop_name  # Name of the crop
        self.soil_n = soil_n  # Current Nitrogen level (kg/ha)
        self.soil_p = soil_p  # Current Phosphorus level (kg/ha)
        self.soil_k = soil_k  # Current Potassium level (kg/ha)
        self.biofertilizer = biofertilizer  # Whether biofertilizer is used

        # Recommended NPK for sugarcane from your data
        self.crop_n_needs = 300  # Recommended Nitrogen (kg/ha)
        self.crop_p_needs = 100  # Recommended Phosphorus (kg/ha)
        self.crop_k_needs = 200  # Recommended Potassium (kg/ha)

        # If biofertilizer is used, reduce nitrogen by 25%
        if self.biofertilizer:
            self.crop_n_needs *= 0.75

        # Initialize fuzzy variables
        self._initialize_fuzzy_system()

        # Growth stages with fertilizer recommendations
        self.growth_stages = {
            "Germination": {"days": (0, 40), "NPK": (100, 33, 66), "biofertilizer": True},  # Organic + Biofertilizer
            "Tillering": {"days": (41, 135), "NPK": (100, 33, 66), "biofertilizer": True},  # Biofertilizer
            "Grand Growth": {"days": (136, 300), "NPK": (100, 34, 68), "biofertilizer": True},
            "Maturity": {"days": (301, 360), "NPK": (0, 0, 0)},  # No additional fertilizer
            "Harvesting": {"days": (360, 400), "NPK": (0, 0, 0)}   # No additional fertilizer
        }

        # NPK complex fertilizers and individual fertilizers
        self.fertilizers = {
            "10:26:26": {"N": 0.10, "P": 0.26, "K": 0.26},
            "12:32:16": {"N": 0.12, "P": 0.32, "K": 0.16},
            "15:15:15": {"N": 0.15, "P": 0.15, "K": 0.15},
            "Urea (46-0-0)": {"N": 0.46, "P": 0.0, "K": 0.0},
            "DAP (18-46-0)": {"N": 0.18, "P": 0.46, "K": 0.0},
            "MOP (0-0-60)": {"N": 0.0, "P": 0.0, "K": 0.60},
            "Ammonium Sulfate (21-0-0)": {"N": 0.21, "P": 0.0, "K": 0.0},
            "CAN (26-0-0)": {"N": 0.26, "P": 0.0, "K": 0.0},
            "SSP (0-16-0)": {"N": 0.0, "P": 0.16, "K": 0.0},
            "TSP (0-44-0)": {"N": 0.0, "P": 0.44, "K": 0.0},
            "SOP (0-0-50)": {"N": 0.0, "P": 0.0, "K": 0.50},
        }

        # Micronutrient management
        self.micronutrients = {
            "Zinc sulphate": {"dose": 37.5, "condition": "zinc_deficient"},
            "Ferrous sulphate": {"dose": 100, "condition": "iron_deficient"},
            "Copper sulphate": {"dose": 5, "condition": "copper_deficient"}
        }

    def _initialize_fuzzy_system(self):
        # Define fuzzy variables
        self.n_needed = ctrl.Antecedent(np.arange(0, 301, 1), 'n_needed')
        self.p_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'p_needed')
        self.k_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'k_needed')

        self.n_fertilizer = ctrl.Consequent(np.arange(0, 301, 1), 'n_fertilizer')
        self.p_fertilizer = ctrl.Consequent(np.arange(0, 101, 1), 'p_fertilizer')
        self.k_fertilizer = ctrl.Consequent(np.arange(0, 201, 1), 'k_fertilizer')

        # Membership functions for inputs (N, P, K needs)
        self.n_needed['low'] = fuzz.trimf(self.n_needed.universe, [0, 0, 150])
        self.n_needed['medium'] = fuzz.trimf(self.n_needed.universe, [100, 150, 200])
        self.n_needed['high'] = fuzz.trimf(self.n_needed.universe, [150, 300, 300])

        self.p_needed['low'] = fuzz.trimf(self.p_needed.universe, [0, 0, 50])
        self.p_needed['medium'] = fuzz.trimf(self.p_needed.universe, [30, 50, 80])
        self.p_needed['high'] = fuzz.trimf(self.p_needed.universe, [50, 100, 100])

        self.k_needed['low'] = fuzz.trimf(self.k_needed.universe, [0, 0, 100])
        self.k_needed['medium'] = fuzz.trimf(self.k_needed.universe, [50, 100, 150])
        self.k_needed['high'] = fuzz.trimf(self.k_needed.universe, [100, 200, 200])

        # Membership functions for outputs (fertilizer)
        self.n_fertilizer['low'] = fuzz.trimf(self.n_fertilizer.universe, [0, 0, 100])
        self.n_fertilizer['medium'] = fuzz.trimf(self.n_fertilizer.universe, [50, 150, 250])
        self.n_fertilizer['high'] = fuzz.trimf(self.n_fertilizer.universe, [150, 300, 300])

        self.p_fertilizer['low'] = fuzz.trimf(self.p_fertilizer.universe, [0, 0, 40])
        self.p_fertilizer['medium'] = fuzz.trimf(self.p_fertilizer.universe, [20, 50, 70])
        self.p_fertilizer['high'] = fuzz.trimf(self.p_fertilizer.universe, [50, 100, 100])

        self.k_fertilizer['low'] = fuzz.trimf(self.k_fertilizer.universe, [0, 0, 50])
        self.k_fertilizer['medium'] = fuzz.trimf(self.k_fertilizer.universe, [30, 100, 150])
        self.k_fertilizer['high'] = fuzz.trimf(self.k_fertilizer.universe, [100, 200, 200])

        # Define fuzzy rules
        self.rules = [
            ctrl.Rule(self.n_needed['low'] & self.p_needed['low'] & self.k_needed['low'],
                      (self.n_fertilizer['low'], self.p_fertilizer['low'], self.k_fertilizer['low'])),
            ctrl.Rule(self.n_needed['medium'] & self.p_needed['medium'] & self.k_needed['medium'],
                      (self.n_fertilizer['medium'], self.p_fertilizer['medium'], self.k_fertilizer['medium'])),
            ctrl.Rule(self.n_needed['high'] & self.p_needed['high'] & self.k_needed['high'],
                      (self.n_fertilizer['high'], self.p_fertilizer['high'], self.k_fertilizer['high'])),
        ]

        # Create control system
        self.fertilizer_ctrl = ctrl.ControlSystem(self.rules)
        self.fertilizer_sim = ctrl.ControlSystemSimulation(self.fertilizer_ctrl)

    def fuzzy_logic(self, growth_stage):
        # Get NPK needs based on growth stage
        n_needed_target, p_needed_target, k_needed_target = self.get_npk_needs(growth_stage)

        # Fuzzy logic to calculate fertilizer required
        n_needed = max(0, n_needed_target - self.soil_n)
        p_needed = max(0, p_needed_target - self.soil_p)
        k_needed = max(0, k_needed_target - self.soil_k)

        # Set inputs to the fuzzy logic system
        self.fertilizer_sim.input['n_needed'] = n_needed
        self.fertilizer_sim.input['p_needed'] = p_needed
        self.fertilizer_sim.input['k_needed'] = k_needed

        # Compute the fuzzy logic output
        self.fertilizer_sim.compute()

        # Get the fertilizer amounts needed
        n_fertilizer_needed = self.fertilizer_sim.output['n_fertilizer']
        p_fertilizer_needed = self.fertilizer_sim.output['p_fertilizer']
        k_fertilizer_needed = self.fertilizer_sim.output['k_fertilizer']

        return n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed

    def get_npk_needs(self, growth_stage):
        # Get the NPK needs based on the growth stage
        return self.growth_stages[growth_stage]['NPK']

    def calculate_fertilizer_amounts(self, n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed):
           amounts = {}
           total_n_applied = total_p_applied = total_k_applied = 0
       
           # Count available fertilizers with N, P, and K to distribute evenly
           n_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["N"] > 0]
           p_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["P"] > 0]
           k_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["K"] > 0]
           
           # Distribute N evenly across all nitrogen-heavy and balanced fertilizers
           for fertilizer in n_fertilizers:
               n_content = self.fertilizers[fertilizer]["N"]
               n_share = n_fertilizer_needed / len(n_fertilizers)  # Split nitrogen need evenly
               n_amount = n_share / n_content if n_content > 0 else 0
               amounts[fertilizer] = amounts.get(fertilizer, 0) + n_amount
               total_n_applied += n_amount * n_content
       
           # Distribute P evenly across all phosphorus-heavy and balanced fertilizers
           for fertilizer in p_fertilizers:
               p_content = self.fertilizers[fertilizer]["P"]
               p_share = p_fertilizer_needed / len(p_fertilizers)  # Split phosphorus need evenly
               p_amount = p_share / p_content if p_content > 0 else 0
               amounts[fertilizer] = amounts.get(fertilizer, 0) + p_amount
               total_p_applied += p_amount * p_content
       
           # Distribute K evenly across all potassium-heavy and balanced fertilizers
           for fertilizer in k_fertilizers:
               k_content = self.fertilizers[fertilizer]["K"]
               k_share = k_fertilizer_needed / len(k_fertilizers)  # Split potassium need evenly
               k_amount = k_share / k_content if k_content > 0 else 0
               amounts[fertilizer] = amounts.get(fertilizer, 0) + k_amount
               total_k_applied += k_amount * k_content
       
           # Rebalance amounts across all fertilizers
           return amounts
       
# # Example of the update in action:
# calculator = NPKComplexFertilizerCalculator_sugarcane(
#     crop_name="Sugarcane", soil_n=500, soil_p=20, soil_k=30, biofertilizer=True
# )
# calculator.display_fertilizer_plan()

      


    def display_fertilizer_plan(self):
        # Initialize a list to store fertilizer plans for each growth stage
        fertilizer_plan_list = []

        print(f"\nFertilizer plan for {self.crop_name}:")
        for phase, stage_data in self.growth_stages.items():
            # Get fertilizer amounts using fuzzy logic
            n_fertilizer, p_fertilizer, k_fertilizer = self.fuzzy_logic(phase)
            fertilizer_amounts = self.calculate_fertilizer_amounts(n_fertilizer, p_fertilizer, k_fertilizer)

            # Display the data
            print(f"{phase} (Days: {stage_data['days'][0]} to {stage_data['days'][1]}):")
            print(f"  N fertilizer needed: {n_fertilizer:.2f} kg/ha")
            print(f"  P fertilizer needed: {p_fertilizer:.2f} kg/ha")
            print(f"  K fertilizer needed: {k_fertilizer:.2f} kg/ha")

            for fertilizer, amount in fertilizer_amounts.items():
                print(f"    {fertilizer}: {amount:.2f} kg/ha")

            print()

            # Store the data for this phase in a dictionary
            phase_data = {
                'phase': phase,
                'days': stage_data['days'],
                'n_fertilizer_needed': n_fertilizer,
                'p_fertilizer_needed': p_fertilizer,
                'k_fertilizer_needed': k_fertilizer,
                'specific_fertilizer_amounts': fertilizer_amounts  # Dictionary with the specific fertilizer amounts
            }

            # Append the dictionary to the list
            fertilizer_plan_list.append(phase_data)

        # Return the list containing all phases' data
        return fertilizer_plan_list


# Example usage
calculator = NPKComplexFertilizerCalculator_sugarcane(crop_name="Sugarcane", soil_n=500, soil_p=20, soil_k=30, biofertilizer=True)
calculator.display_fertilizer_plan()



Fertilizer plan for Sugarcane:
Germination (Days: 0 to 40):
  N fertilizer needed: 36.51 kg/ha
  P fertilizer needed: 14.60 kg/ha
  K fertilizer needed: 18.25 kg/ha
    10:26:26: 75.56 kg/ha
    12:32:16: 73.89 kg/ha
    15:15:15: 75.34 kg/ha
    Urea (46-0-0): 11.34 kg/ha
    DAP (18-46-0): 34.27 kg/ha
    Ammonium Sulfate (21-0-0): 24.84 kg/ha
    CAN (26-0-0): 20.06 kg/ha
    SSP (0-16-0): 15.21 kg/ha
    TSP (0-44-0): 5.53 kg/ha
    MOP (0-0-60): 6.08 kg/ha
    SOP (0-0-50): 7.30 kg/ha

Tillering (Days: 41 to 135):
  N fertilizer needed: 36.51 kg/ha
  P fertilizer needed: 14.60 kg/ha
  K fertilizer needed: 18.25 kg/ha
    10:26:26: 75.56 kg/ha
    12:32:16: 73.89 kg/ha
    15:15:15: 75.34 kg/ha
    Urea (46-0-0): 11.34 kg/ha
    DAP (18-46-0): 34.27 kg/ha
    Ammonium Sulfate (21-0-0): 24.84 kg/ha
    CAN (26-0-0): 20.06 kg/ha
    SSP (0-16-0): 15.21 kg/ha
    TSP (0-44-0): 5.53 kg/ha
    MOP (0-0-60): 6.08 kg/ha
    SOP (0-0-50): 7.30 kg/ha

Grand Growth (Days: 136 to 300):
  N f

[{'phase': 'Germination',
  'days': (0, 40),
  'n_fertilizer_needed': 36.50980392156863,
  'p_fertilizer_needed': 14.60392156862745,
  'k_fertilizer_needed': 18.25490196078431,
  'specific_fertilizer_amounts': {'10:26:26': 75.56058320764204,
   '12:32:16': 73.88888888888889,
   '15:15:15': 75.33769063180829,
   'Urea (46-0-0)': 11.338448422847401,
   'DAP (18-46-0)': 34.26731078904992,
   'Ammonium Sulfate (21-0-0)': 24.836601307189547,
   'CAN (26-0-0)': 20.06033182503771,
   'SSP (0-16-0)': 15.212418300653594,
   'TSP (0-44-0)': 5.531788472964943,
   'MOP (0-0-60)': 6.084967320261437,
   'SOP (0-0-50)': 7.3019607843137235}},
 {'phase': 'Tillering',
  'days': (41, 135),
  'n_fertilizer_needed': 36.50980392156863,
  'p_fertilizer_needed': 14.60392156862745,
  'k_fertilizer_needed': 18.25490196078431,
  'specific_fertilizer_amounts': {'10:26:26': 75.56058320764204,
   '12:32:16': 73.88888888888889,
   '15:15:15': 75.33769063180829,
   'Urea (46-0-0)': 11.338448422847401,
   'DAP (18-46-

In [6]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class NPKComplexFertilizerCalculator_maize:
    def __init__(self, crop_name, soil_n, soil_p, soil_k, biofertilizer=False):
        self.crop_name = crop_name  # Name of the crop
        self.soil_n = soil_n  # Current Nitrogen level (kg/ha)
        self.soil_p = soil_p  # Current Phosphorus level (kg/ha)
        self.soil_k = soil_k  # Current Potassium level (kg/ha)
        self.biofertilizer = biofertilizer  # Whether biofertilizer is used

        # Recommended NPK for maize from standard agricultural practices
        self.crop_n_needs = 150  # Recommended Nitrogen (kg/ha)
        self.crop_p_needs = 60   # Recommended Phosphorus (kg/ha)
        self.crop_k_needs = 80   # Recommended Potassium (kg/ha)

        # If biofertilizer is used, reduce nitrogen by 25%
        if self.biofertilizer:
            self.crop_n_needs *= 0.75

        # Initialize fuzzy variables
        self._initialize_fuzzy_system()

        # Growth stages with fertilizer recommendations for maize
        self.growth_stages = {
            "Seedling": {"days": (0, 20), "NPK": (30, 40, 20), "biofertilizer": True},  # Initial nutrients
            "Vegetative": {"days": (21, 55), "NPK": (60, 20, 30), "biofertilizer": True},  # Rapid growth phase
            "Tasseling": {"days": (56, 75), "NPK": (40, 0, 20), "biofertilizer": False},  # Critical stage
            "Grain Filling": {"days": (76, 105), "NPK": (20, 0, 10), "biofertilizer": False},  # Yield formation
            "Maturity": {"days": (106, 130), "NPK": (0, 0, 0)}  # No additional fertilizer
        }

        # NPK complex fertilizers and individual fertilizers
        self.fertilizers = {
            "10:26:26": {"N": 0.10, "P": 0.26, "K": 0.26},
            "12:32:16": {"N": 0.12, "P": 0.32, "K": 0.16},
            "15:15:15": {"N": 0.15, "P": 0.15, "K": 0.15},
            "Urea (46-0-0)": {"N": 0.46, "P": 0.0, "K": 0.0},
            "DAP (18-46-0)": {"N": 0.18, "P": 0.46, "K": 0.0},
            "MOP (0-0-60)": {"N": 0.0, "P": 0.0, "K": 0.60},
            "Ammonium Sulfate (21-0-0)": {"N": 0.21, "P": 0.0, "K": 0.0},
            "SSP (0-16-0)": {"N": 0.0, "P": 0.16, "K": 0.0},
            "TSP (0-44-0)": {"N": 0.0, "P": 0.44, "K": 0.0},
            "SOP (0-0-50)": {"N": 0.0, "P": 0.0, "K": 0.50}
        }

        # Micronutrient management specific to maize
        self.micronutrients = {
            "Zinc sulphate": {"dose": 25, "condition": "zinc_deficient"},
            "Ferrous sulphate": {"dose": 50, "condition": "iron_deficient"},
            "Boron": {"dose": 5, "condition": "boron_deficient"}
        }

    def _initialize_fuzzy_system(self):
        # Define fuzzy variables with ranges appropriate for maize
        self.n_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'n_needed')
        self.p_needed = ctrl.Antecedent(np.arange(0, 101, 1), 'p_needed')
        self.k_needed = ctrl.Antecedent(np.arange(0, 101, 1), 'k_needed')

        self.n_fertilizer = ctrl.Consequent(np.arange(0, 201, 1), 'n_fertilizer')
        self.p_fertilizer = ctrl.Consequent(np.arange(0, 101, 1), 'p_fertilizer')
        self.k_fertilizer = ctrl.Consequent(np.arange(0, 101, 1), 'k_fertilizer')

        # Membership functions for inputs (N, P, K needs) - adjusted for maize
        self.n_needed['low'] = fuzz.trimf(self.n_needed.universe, [0, 0, 75])
        self.n_needed['medium'] = fuzz.trimf(self.n_needed.universe, [50, 100, 150])
        self.n_needed['high'] = fuzz.trimf(self.n_needed.universe, [100, 200, 200])

        self.p_needed['low'] = fuzz.trimf(self.p_needed.universe, [0, 0, 30])
        self.p_needed['medium'] = fuzz.trimf(self.p_needed.universe, [20, 40, 60])
        self.p_needed['high'] = fuzz.trimf(self.p_needed.universe, [40, 100, 100])

        self.k_needed['low'] = fuzz.trimf(self.k_needed.universe, [0, 0, 40])
        self.k_needed['medium'] = fuzz.trimf(self.k_needed.universe, [30, 50, 70])
        self.k_needed['high'] = fuzz.trimf(self.k_needed.universe, [50, 100, 100])

        # Membership functions for outputs (fertilizer) - adjusted for maize
        self.n_fertilizer['low'] = fuzz.trimf(self.n_fertilizer.universe, [0, 0, 50])
        self.n_fertilizer['medium'] = fuzz.trimf(self.n_fertilizer.universe, [30, 75, 125])
        self.n_fertilizer['high'] = fuzz.trimf(self.n_fertilizer.universe, [100, 200, 200])

        self.p_fertilizer['low'] = fuzz.trimf(self.p_fertilizer.universe, [0, 0, 20])
        self.p_fertilizer['medium'] = fuzz.trimf(self.p_fertilizer.universe, [15, 30, 45])
        self.p_fertilizer['high'] = fuzz.trimf(self.p_fertilizer.universe, [30, 100, 100])

        self.k_fertilizer['low'] = fuzz.trimf(self.k_fertilizer.universe, [0, 0, 30])
        self.k_fertilizer['medium'] = fuzz.trimf(self.k_fertilizer.universe, [20, 40, 60])
        self.k_fertilizer['high'] = fuzz.trimf(self.k_fertilizer.universe, [40, 100, 100])

        # Define fuzzy rules
        self.rules = [
            ctrl.Rule(self.n_needed['low'] & self.p_needed['low'] & self.k_needed['low'],
                      (self.n_fertilizer['low'], self.p_fertilizer['low'], self.k_fertilizer['low'])),
            ctrl.Rule(self.n_needed['medium'] & self.p_needed['medium'] & self.k_needed['medium'],
                      (self.n_fertilizer['medium'], self.p_fertilizer['medium'], self.k_fertilizer['medium'])),
            ctrl.Rule(self.n_needed['high'] & self.p_needed['high'] & self.k_needed['high'],
                      (self.n_fertilizer['high'], self.p_fertilizer['high'], self.k_fertilizer['high'])),
        ]

        # Create control system
        self.fertilizer_ctrl = ctrl.ControlSystem(self.rules)
        self.fertilizer_sim = ctrl.ControlSystemSimulation(self.fertilizer_ctrl)

    def fuzzy_logic(self, growth_stage):
        # Get NPK needs based on growth stage
        n_needed_target, p_needed_target, k_needed_target = self.get_npk_needs(growth_stage)

        # Fuzzy logic to calculate fertilizer required
        n_needed = max(0, n_needed_target - self.soil_n)
        p_needed = max(0, p_needed_target - self.soil_p)
        k_needed = max(0, k_needed_target - self.soil_k)

        # Set inputs to the fuzzy logic system
        self.fertilizer_sim.input['n_needed'] = n_needed
        self.fertilizer_sim.input['p_needed'] = p_needed
        self.fertilizer_sim.input['k_needed'] = k_needed

        # Compute the fuzzy logic output
        self.fertilizer_sim.compute()

        # Get the fertilizer amounts needed
        n_fertilizer_needed = self.fertilizer_sim.output['n_fertilizer']
        p_fertilizer_needed = self.fertilizer_sim.output['p_fertilizer']
        k_fertilizer_needed = self.fertilizer_sim.output['k_fertilizer']

        return n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed

    def get_npk_needs(self, growth_stage):
        # Get the NPK needs based on the growth stage
        return self.growth_stages[growth_stage]['NPK']

    def calculate_fertilizer_amounts(self, n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed):
        amounts = {}
        total_n_applied = total_p_applied = total_k_applied = 0
    
        # Count available fertilizers with N, P, and K to distribute evenly
        n_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["N"] > 0]
        p_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["P"] > 0]
        k_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["K"] > 0]
        
        # Distribute N evenly across all nitrogen-heavy and balanced fertilizers
        for fertilizer in n_fertilizers:
            n_content = self.fertilizers[fertilizer]["N"]
            n_share = n_fertilizer_needed / len(n_fertilizers)  # Split nitrogen need evenly
            n_amount = n_share / n_content if n_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + n_amount
            total_n_applied += n_amount * n_content
    
        # Distribute P evenly across all phosphorus-heavy and balanced fertilizers
        for fertilizer in p_fertilizers:
            p_content = self.fertilizers[fertilizer]["P"]
            p_share = p_fertilizer_needed / len(p_fertilizers)  # Split phosphorus need evenly
            p_amount = p_share / p_content if p_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + p_amount
            total_p_applied += p_amount * p_content
    
        # Distribute K evenly across all potassium-heavy and balanced fertilizers
        for fertilizer in k_fertilizers:
            k_content = self.fertilizers[fertilizer]["K"]
            k_share = k_fertilizer_needed / len(k_fertilizers)  # Split potassium need evenly
            k_amount = k_share / k_content if k_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + k_amount
            total_k_applied += k_amount * k_content
    
        return amounts

    def display_fertilizer_plan(self):
        fertilizer_plan_list = []

        print(f"\nFertilizer plan for {self.crop_name}:")
        for phase, stage_data in self.growth_stages.items():
            # Get fertilizer amounts using fuzzy logic
            n_fertilizer, p_fertilizer, k_fertilizer = self.fuzzy_logic(phase)
            fertilizer_amounts = self.calculate_fertilizer_amounts(n_fertilizer, p_fertilizer, k_fertilizer)

            # Display the data
            print(f"{phase} (Days: {stage_data['days'][0]} to {stage_data['days'][1]}):")
            print(f"  N fertilizer needed: {n_fertilizer:.2f} kg/ha")
            print(f"  P fertilizer needed: {p_fertilizer:.2f} kg/ha")
            print(f"  K fertilizer needed: {k_fertilizer:.2f} kg/ha")

            for fertilizer, amount in fertilizer_amounts.items():
                print(f"    {fertilizer}: {amount:.2f} kg/ha")

            print()

            # Store the data for this phase
            phase_data = {
                'phase': phase,
                'days': stage_data['days'],
                'n_fertilizer_needed': n_fertilizer,
                'p_fertilizer_needed': p_fertilizer,
                'k_fertilizer_needed': k_fertilizer,
                'specific_fertilizer_amounts': fertilizer_amounts
            }

            fertilizer_plan_list.append(phase_data)

        return fertilizer_plan_list

# Example usage
calculator = NPKComplexFertilizerCalculator_maize(
    crop_name="Maize", soil_n=40, soil_p=15, soil_k=20, biofertilizer=True
)
calculator.display_fertilizer_plan()


Fertilizer plan for Maize:
Seedling (Days: 0 to 20):
  N fertilizer needed: 22.98 kg/ha
  P fertilizer needed: 9.19 kg/ha
  K fertilizer needed: 13.79 kg/ha
    10:26:26: 54.80 kg/ha
    12:32:16: 53.94 kg/ha
    15:15:15: 54.13 kg/ha
    Urea (46-0-0): 8.33 kg/ha
    DAP (18-46-0): 24.61 kg/ha
    Ammonium Sulfate (21-0-0): 18.24 kg/ha
    SSP (0-16-0): 9.57 kg/ha
    TSP (0-44-0): 3.48 kg/ha
    MOP (0-0-60): 4.60 kg/ha
    SOP (0-0-50): 5.52 kg/ha

Vegetative (Days: 21 to 55):
  N fertilizer needed: 17.60 kg/ha
  P fertilizer needed: 7.04 kg/ha
  K fertilizer needed: 10.56 kg/ha
    10:26:26: 41.97 kg/ha
    12:32:16: 41.32 kg/ha
    15:15:15: 41.46 kg/ha
    Urea (46-0-0): 6.38 kg/ha
    DAP (18-46-0): 18.85 kg/ha
    Ammonium Sulfate (21-0-0): 13.97 kg/ha
    SSP (0-16-0): 7.33 kg/ha
    TSP (0-44-0): 2.67 kg/ha
    MOP (0-0-60): 3.52 kg/ha
    SOP (0-0-50): 4.22 kg/ha

Tasseling (Days: 56 to 75):
  N fertilizer needed: 16.67 kg/ha
  P fertilizer needed: 6.67 kg/ha
  K fertilizer

[{'phase': 'Seedling',
  'days': (0, 20),
  'n_fertilizer_needed': 22.979797979797976,
  'p_fertilizer_needed': 9.191919191919194,
  'k_fertilizer_needed': 13.787878787878794,
  'specific_fertilizer_amounts': {'10:26:26': 54.79797979797979,
   '12:32:16': 53.938692480359144,
   '15:15:15': 54.13019079685746,
   'Urea (46-0-0)': 8.326013760796368,
   'DAP (18-46-0)': 24.607996226353713,
   'Ammonium Sulfate (21-0-0)': 18.237934904601566,
   'SSP (0-16-0)': 9.574915824915827,
   'TSP (0-44-0)': 3.481787572696664,
   'MOP (0-0-60)': 4.595959595959598,
   'SOP (0-0-50)': 5.515151515151517}},
 {'phase': 'Vegetative',
  'days': (21, 55),
  'n_fertilizer_needed': 17.602339181286546,
  'p_fertilizer_needed': 7.04093567251462,
  'k_fertilizer_needed': 10.561403508771926,
  'specific_fertilizer_amounts': {'10:26:26': 41.97480881691407,
   '12:32:16': 41.3166016894087,
   '15:15:15': 41.46328784925276,
   'Urea (46-0-0)': 6.377659123654546,
   'DAP (18-46-0)': 18.84952585435677,
   'Ammonium Sulf

In [5]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class NPKComplexFertilizerCalculator_grapes:
    def __init__(self, crop_name, soil_n, soil_p, soil_k, vine_age=None, biofertilizer=False):
        self.crop_name = crop_name  # Name of the crop
        self.soil_n = soil_n  # Current N/itrogen level (kg/ha)
        self.soil_p = soil_p  # Current Phosphorus level (kg/ha)
        self.soil_k = soil_k  # Current Potassium level (kg/ha)
        self.vine_age = vine_age  # Age of vines in years
        self.biofertilizer = biofertilizer  # Whether biofertilizer is used

        # Base NPK recommendations for grapes (kg/ha)
        # These values are for mature vines (4+ years)
        self.crop_n_needs = 120  # Recommended Nitrogen
        self.crop_p_needs = 50   # Recommended Phosphorus
        self.crop_k_needs = 150  # Recommended Potassium

        # Adjust nutrients based on vine age
        if vine_age is not None:
            self._adjust_npk_for_vine_age()

        # If biofertilizer is used, reduce nitrogen by 25%
        if self.biofertilizer:
            self.crop_n_needs *= 0.75

        # Initialize fuzzy variables
        self._initialize_fuzzy_system()

        # Growth stages with fertilizer recommendations for grapes
        self.growth_stages = {
            "Bud Break": {
                "days": (0, 30),
                "NPK": (20, 30, 20),
                "biofertilizer": True,
                "notes": "Early spring application"
            },
            "Flowering": {
                "days": (31, 60),
                "NPK": (30, 10, 30),
                "biofertilizer": True,
                "notes": "Critical for fruit set"
            },
            "Fruit Set": {
                "days": (61, 90),
                "NPK": (25, 5, 40),
                "biofertilizer": False,
                "notes": "Higher K requirement"
            },
            "Berry Development": {
                "days": (91, 120),
                "NPK": (25, 5, 40),
                "biofertilizer": False,
                "notes": "Continue K application"
            },
            "Veraison": {
                "days": (121, 150),
                "NPK": (10, 0, 20),
                "biofertilizer": False,
                "notes": "Reduce N, maintain K"
            },
            "Harvest": {
                "days": (151, 180),
                "NPK": (10, 0, 0),
                "biofertilizer": False,
                "notes": "Minimal fertilization"
            }
        }

        # NPK complex fertilizers and individual fertilizers
        self.fertilizers = {
            "10:26:26": {"N": 0.10, "P": 0.26, "K": 0.26},
            "12:32:16": {"N": 0.12, "P": 0.32, "K": 0.16},
            "15:15:15": {"N": 0.15, "P": 0.15, "K": 0.15},
            "Urea (46-0-0)": {"N": 0.46, "P": 0.0, "K": 0.0},
            "DAP (18-46-0)": {"N": 0.18, "P": 0.46, "K": 0.0},
            "MOP (0-0-60)": {"N": 0.0, "P": 0.0, "K": 0.60},
            "SOP (0-0-50)": {"N": 0.0, "P": 0.0, "K": 0.50},  # Preferred K source for grapes
            "SSP (0-16-0)": {"N": 0.0, "P": 0.16, "K": 0.0},
            "Calcium Nitrate": {"N": 0.155, "P": 0.0, "K": 0.0},  # Added for grape nutrition
            "Magnesium Sulfate": {"N": 0.0, "P": 0.0, "K": 0.0}   # Secondary nutrient
        }

        # Micronutrient management specific to grapes
        self.micronutrients = {
            "Zinc sulphate": {"dose": 15, "condition": "zinc_deficient"},
            "Ferrous sulphate": {"dose": 25, "condition": "iron_deficient"},
            "Boron": {"dose": 3, "condition": "boron_deficient"},
            "Magnesium sulphate": {"dose": 20, "condition": "magnesium_deficient"}
        }

    def _adjust_npk_for_vine_age(self):
        """Adjust NPK needs based on vine age"""
        if self.vine_age < 1:
            # Young vines (< 1 year)
            self.crop_n_needs *= 0.3
            self.crop_p_needs *= 0.4
            self.crop_k_needs *= 0.3
        elif self.vine_age < 2:
            # 1-2 year old vines
            self.crop_n_needs *= 0.5
            self.crop_p_needs *= 0.6
            self.crop_k_needs *= 0.5
        elif self.vine_age < 4:
            # 2-4 year old vines
            self.crop_n_needs *= 0.75
            self.crop_p_needs *= 0.8
            self.crop_k_needs *= 0.75

    def _initialize_fuzzy_system(self):
        # Define fuzzy variables with ranges appropriate for grapes
        self.n_needed = ctrl.Antecedent(np.arange(0, 151, 1), 'n_needed')
        self.p_needed = ctrl.Antecedent(np.arange(0, 101, 1), 'p_needed')
        self.k_needed = ctrl.Antecedent(np.arange(0, 201, 1), 'k_needed')

        self.n_fertilizer = ctrl.Consequent(np.arange(0, 151, 1), 'n_fertilizer')
        self.p_fertilizer = ctrl.Consequent(np.arange(0, 101, 1), 'p_fertilizer')
        self.k_fertilizer = ctrl.Consequent(np.arange(0, 201, 1), 'k_fertilizer')

        # Membership functions for inputs (N, P, K needs)
        self.n_needed['low'] = fuzz.trimf(self.n_needed.universe, [0, 0, 60])
        self.n_needed['medium'] = fuzz.trimf(self.n_needed.universe, [40, 80, 120])
        self.n_needed['high'] = fuzz.trimf(self.n_needed.universe, [80, 150, 150])

        self.p_needed['low'] = fuzz.trimf(self.p_needed.universe, [0, 0, 25])
        self.p_needed['medium'] = fuzz.trimf(self.p_needed.universe, [15, 35, 55])
        self.p_needed['high'] = fuzz.trimf(self.p_needed.universe, [35, 100, 100])

        self.k_needed['low'] = fuzz.trimf(self.k_needed.universe, [0, 0, 75])
        self.k_needed['medium'] = fuzz.trimf(self.k_needed.universe, [50, 100, 150])
        self.k_needed['high'] = fuzz.trimf(self.k_needed.universe, [100, 200, 200])

        # Membership functions for outputs (fertilizer)
        self.n_fertilizer['low'] = fuzz.trimf(self.n_fertilizer.universe, [0, 0, 40])
        self.n_fertilizer['medium'] = fuzz.trimf(self.n_fertilizer.universe, [20, 60, 100])
        self.n_fertilizer['high'] = fuzz.trimf(self.n_fertilizer.universe, [60, 150, 150])

        self.p_fertilizer['low'] = fuzz.trimf(self.p_fertilizer.universe, [0, 0, 20])
        self.p_fertilizer['medium'] = fuzz.trimf(self.p_fertilizer.universe, [10, 30, 50])
        self.p_fertilizer['high'] = fuzz.trimf(self.p_fertilizer.universe, [30, 100, 100])

        self.k_fertilizer['low'] = fuzz.trimf(self.k_fertilizer.universe, [0, 0, 50])
        self.k_fertilizer['medium'] = fuzz.trimf(self.k_fertilizer.universe, [30, 75, 120])
        self.k_fertilizer['high'] = fuzz.trimf(self.k_fertilizer.universe, [75, 200, 200])

        # Define fuzzy rules
        self.rules = [
            ctrl.Rule(self.n_needed['low'] & self.p_needed['low'] & self.k_needed['low'],
                      (self.n_fertilizer['low'], self.p_fertilizer['low'], self.k_fertilizer['low'])),
            ctrl.Rule(self.n_needed['medium'] & self.p_needed['medium'] & self.k_needed['medium'],
                      (self.n_fertilizer['medium'], self.p_fertilizer['medium'], self.k_fertilizer['medium'])),
            ctrl.Rule(self.n_needed['high'] & self.p_needed['high'] & self.k_needed['high'],
                      (self.n_fertilizer['high'], self.p_fertilizer['high'], self.k_fertilizer['high'])),
        ]

        # Create control system
        self.fertilizer_ctrl = ctrl.ControlSystem(self.rules)
        self.fertilizer_sim = ctrl.ControlSystemSimulation(self.fertilizer_ctrl)

    def fuzzy_logic(self, growth_stage):
        # Get NPK needs based on growth stage
        n_needed_target, p_needed_target, k_needed_target = self.get_npk_needs(growth_stage)

        # Fuzzy logic to calculate fertilizer required
        n_needed = max(0, n_needed_target - self.soil_n)
        p_needed = max(0, p_needed_target - self.soil_p)
        k_needed = max(0, k_needed_target - self.soil_k)

        # Set inputs to the fuzzy logic system
        self.fertilizer_sim.input['n_needed'] = n_needed
        self.fertilizer_sim.input['p_needed'] = p_needed
        self.fertilizer_sim.input['k_needed'] = k_needed

        # Compute the fuzzy logic output
        self.fertilizer_sim.compute()

        # Get the fertilizer amounts needed
        n_fertilizer_needed = self.fertilizer_sim.output['n_fertilizer']
        p_fertilizer_needed = self.fertilizer_sim.output['p_fertilizer']
        k_fertilizer_needed = self.fertilizer_sim.output['k_fertilizer']

        return n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed

    def get_npk_needs(self, growth_stage):
        # Get the NPK needs based on the growth stage
        return self.growth_stages[growth_stage]['NPK']

    def calculate_fertilizer_amounts(self, n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed):
        amounts = {}
        total_n_applied = total_p_applied = total_k_applied = 0
    
        # Prioritize SOP over MOP for grapes
        k_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["K"] > 0]
        if "SOP (0-0-50)" in k_fertilizers:
            k_fertilizers.remove("SOP (0-0-50)")
            k_fertilizers.insert(0, "SOP (0-0-50)")  # Put SOP first
    
        # Count available fertilizers with N, P, and K
        n_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["N"] > 0]
        p_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["P"] > 0]
        
        # Distribute nutrients with preference for grape-friendly fertilizers
        for fertilizer in n_fertilizers:
            n_content = self.fertilizers[fertilizer]["N"]
            n_share = n_fertilizer_needed / len(n_fertilizers)
            n_amount = n_share / n_content if n_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + n_amount
            total_n_applied += n_amount * n_content
    
        for fertilizer in p_fertilizers:
            p_content = self.fertilizers[fertilizer]["P"]
            p_share = p_fertilizer_needed / len(p_fertilizers)
            p_amount = p_share / p_content if p_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + p_amount
            total_p_applied += p_amount * p_content
    
        for fertilizer in k_fertilizers:
            k_content = self.fertilizers[fertilizer]["K"]
            k_share = k_fertilizer_needed / len(k_fertilizers)
            k_amount = k_share / k_content if k_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + k_amount
            total_k_applied += k_amount * k_content
    
        return amounts

    def display_fertilizer_plan(self):
        fertilizer_plan_list = []

        print(f"\nFertilizer plan for {self.crop_name}:")
        if self.vine_age is not None:
            print(f"Vine age: {self.vine_age} years")
            
        for phase, stage_data in self.growth_stages.items():
            # Get fertilizer amounts using fuzzy logic
            n_fertilizer, p_fertilizer, k_fertilizer = self.fuzzy_logic(phase)
            fertilizer_amounts = self.calculate_fertilizer_amounts(n_fertilizer, p_fertilizer, k_fertilizer)

            # Display the data
            print(f"\n{phase} (Days: {stage_data['days'][0]} to {stage_data['days'][1]}):")
            print(f"Notes: {stage_data['notes']}")
            print(f"  N fertilizer needed: {n_fertilizer:.2f} kg/ha")
            print(f"  P fertilizer needed: {p_fertilizer:.2f} kg/ha")
            print(f"  K fertilizer needed: {k_fertilizer:.2f} kg/ha")

            for fertilizer, amount in fertilizer_amounts.items():
                print(f"    {fertilizer}: {amount:.2f} kg/ha")
calculator = NPKComplexFertilizerCalculator_grapes(
    crop_name="Maize", soil_n=40, soil_p=15, soil_k=20, biofertilizer=True
)
calculator.display_fertilizer_plan()
            # Store the data for this


Fertilizer plan for Maize:

Bud Break (Days: 0 to 30):
Notes: Early spring application
  N fertilizer needed: 16.33 kg/ha
  P fertilizer needed: 8.17 kg/ha
  K fertilizer needed: 20.42 kg/ha
    10:26:26: 49.21 kg/ha
    12:32:16: 53.31 kg/ha
    15:15:15: 56.26 kg/ha
    Urea (46-0-0): 5.92 kg/ha
    DAP (18-46-0): 18.67 kg/ha
    Calcium Nitrate: 17.56 kg/ha
    SSP (0-16-0): 10.21 kg/ha
    SOP (0-0-50): 8.17 kg/ha
    MOP (0-0-60): 6.81 kg/ha

Flowering (Days: 31 to 60):
Notes: Critical for fruit set
  N fertilizer needed: 13.54 kg/ha
  P fertilizer needed: 6.77 kg/ha
  K fertilizer needed: 16.93 kg/ha
    10:26:26: 40.80 kg/ha
    12:32:16: 44.20 kg/ha
    15:15:15: 46.65 kg/ha
    Urea (46-0-0): 4.91 kg/ha
    DAP (18-46-0): 15.48 kg/ha
    Calcium Nitrate: 14.56 kg/ha
    SSP (0-16-0): 8.46 kg/ha
    SOP (0-0-50): 6.77 kg/ha
    MOP (0-0-60): 5.64 kg/ha

Fruit Set (Days: 61 to 90):
Notes: Higher K requirement
  N fertilizer needed: 14.08 kg/ha
  P fertilizer needed: 7.04 kg/ha


In [3]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class NPKComplexFertilizerCalculator_rice:
    def __init__(self, crop_name, soil_n, soil_p, soil_k, biofertilizer=False):
        self.crop_name = crop_name  # Name of the crop
        self.soil_n = soil_n  # Current Nitrogen level (kg/ha)
        self.soil_p = soil_p  # Current Phosphorus level (kg/ha)
        self.soil_k = soil_k  # Current Potassium level (kg/ha)
        self.biofertilizer = biofertilizer  # Whether biofertilizer is used

        # Recommended NPK for rice from standard agricultural practices
        self.crop_n_needs = 120  # Recommended Nitrogen (kg/ha)
        self.crop_p_needs = 40   # Recommended Phosphorus (kg/ha)
        self.crop_k_needs = 60   # Recommended Potassium (kg/ha)

        # If biofertilizer is used, reduce nitrogen by 25%
        if self.biofertilizer:
            self.crop_n_needs *= 0.75

        # Initialize fuzzy variables
        self._initialize_fuzzy_system()

        # Growth stages with fertilizer recommendations for rice
        self.growth_stages = {
            "Seedling": {"days": (0, 21), "NPK": (20, 50, 20), "biofertilizer": True},  # Initial nutrients
            "Tillering": {"days": (22, 45), "NPK": (40, 30, 20), "biofertilizer": True},  # Active tillering
            "Panicle Initiation": {"days": (46, 65), "NPK": (40, 20, 30), "biofertilizer": False},  # Critical stage
            "Heading": {"days": (66, 85), "NPK": (20, 0, 20), "biofertilizer": False},  # Grain formation
            "Ripening": {"days": (86, 110), "NPK": (0, 0, 10)}  # Maturity phase
        }

        # NPK complex fertilizers and individual fertilizers
        self.fertilizers = {
            "10:26:26": {"N": 0.10, "P": 0.26, "K": 0.26},
            "12:32:16": {"N": 0.12, "P": 0.32, "K": 0.16},
            "15:15:15": {"N": 0.15, "P": 0.15, "K": 0.15},
            "Urea (46-0-0)": {"N": 0.46, "P": 0.0, "K": 0.0},
            "DAP (18-46-0)": {"N": 0.18, "P": 0.46, "K": 0.0},
            "MOP (0-0-60)": {"N": 0.0, "P": 0.0, "K": 0.60},
            "Ammonium Sulfate (21-0-0)": {"N": 0.21, "P": 0.0, "K": 0.0},
            "SSP (0-16-0)": {"N": 0.0, "P": 0.16, "K": 0.0}
        }

        # Micronutrient management specific to rice
        self.micronutrients = {
            "Zinc sulphate": {"dose": 25, "condition": "zinc_deficient"},
            "Ferrous sulphate": {"dose": 50, "condition": "iron_deficient"},
            "Manganese sulphate": {"dose": 15, "condition": "manganese_deficient"}
        }

    def _initialize_fuzzy_system(self):
        # Define fuzzy variables with ranges appropriate for rice
        self.n_needed = ctrl.Antecedent(np.arange(0, 151, 1), 'n_needed')
        self.p_needed = ctrl.Antecedent(np.arange(0, 81, 1), 'p_needed')
        self.k_needed = ctrl.Antecedent(np.arange(0, 81, 1), 'k_needed')

        self.n_fertilizer = ctrl.Consequent(np.arange(0, 151, 1), 'n_fertilizer')
        self.p_fertilizer = ctrl.Consequent(np.arange(0, 81, 1), 'p_fertilizer')
        self.k_fertilizer = ctrl.Consequent(np.arange(0, 81, 1), 'k_fertilizer')

        # Membership functions for inputs (N, P, K needs) - adjusted for rice
        self.n_needed['low'] = fuzz.trimf(self.n_needed.universe, [0, 0, 60])
        self.n_needed['medium'] = fuzz.trimf(self.n_needed.universe, [40, 80, 120])
        self.n_needed['high'] = fuzz.trimf(self.n_needed.universe, [80, 150, 150])

        self.p_needed['low'] = fuzz.trimf(self.p_needed.universe, [0, 0, 20])
        self.p_needed['medium'] = fuzz.trimf(self.p_needed.universe, [15, 30, 45])
        self.p_needed['high'] = fuzz.trimf(self.p_needed.universe, [30, 80, 80])

        self.k_needed['low'] = fuzz.trimf(self.k_needed.universe, [0, 0, 30])
        self.k_needed['medium'] = fuzz.trimf(self.k_needed.universe, [20, 40, 60])
        self.k_needed['high'] = fuzz.trimf(self.k_needed.universe, [40, 80, 80])

        # Membership functions for outputs (fertilizer) - adjusted for rice
        self.n_fertilizer['low'] = fuzz.trimf(self.n_fertilizer.universe, [0, 0, 40])
        self.n_fertilizer['medium'] = fuzz.trimf(self.n_fertilizer.universe, [30, 60, 90])
        self.n_fertilizer['high'] = fuzz.trimf(self.n_fertilizer.universe, [70, 150, 150])

        self.p_fertilizer['low'] = fuzz.trimf(self.p_fertilizer.universe, [0, 0, 15])
        self.p_fertilizer['medium'] = fuzz.trimf(self.p_fertilizer.universe, [10, 25, 40])
        self.p_fertilizer['high'] = fuzz.trimf(self.p_fertilizer.universe, [30, 80, 80])

        self.k_fertilizer['low'] = fuzz.trimf(self.k_fertilizer.universe, [0, 0, 20])
        self.k_fertilizer['medium'] = fuzz.trimf(self.k_fertilizer.universe, [15, 30, 45])
        self.k_fertilizer['high'] = fuzz.trimf(self.k_fertilizer.universe, [35, 80, 80])

        # Define fuzzy rules
        self.rules = [
            ctrl.Rule(self.n_needed['low'] & self.p_needed['low'] & self.k_needed['low'],
              (self.n_fertilizer['low'], self.p_fertilizer['low'], self.k_fertilizer['low'])),
    ctrl.Rule(self.n_needed['medium'] & self.p_needed['medium'] & self.k_needed['medium'],
              (self.n_fertilizer['medium'], self.p_fertilizer['medium'], self.k_fertilizer['medium'])),
    ctrl.Rule(self.n_needed['high'] & self.p_needed['high'] & self.k_needed['high'],
              (self.n_fertilizer['high'], self.p_fertilizer['high'], self.k_fertilizer['high'])),

    # Catch-all rule for when inputs don't match any specific rule
    ctrl.Rule(self.n_needed['low'] | self.n_needed['medium'] | self.n_needed['high'],
              self.n_fertilizer['medium']),
    ctrl.Rule(self.p_needed['low'] | self.p_needed['medium'] | self.p_needed['high'],
              self.p_fertilizer['medium']),
    ctrl.Rule(self.k_needed['low'] | self.k_needed['medium'] | self.k_needed['high'],
              self.k_fertilizer['medium']),
        ]

        # Create control system
        self.fertilizer_ctrl = ctrl.ControlSystem(self.rules)
        self.fertilizer_sim = ctrl.ControlSystemSimulation(self.fertilizer_ctrl)

    def fuzzy_logic(self, growth_stage):
        # Get NPK needs based on growth stage
        n_needed_target, p_needed_target, k_needed_target = self.get_npk_needs(growth_stage)

        # Fuzzy logic to calculate fertilizer required
        n_needed = max(0, n_needed_target - self.soil_n)
        p_needed = max(0, p_needed_target - self.soil_p)
        k_needed = max(0, k_needed_target - self.soil_k)

        # Set inputs to the fuzzy logic system
        self.fertilizer_sim.input['n_needed'] = n_needed
        self.fertilizer_sim.input['p_needed'] = p_needed
        self.fertilizer_sim.input['k_needed'] = k_needed

        # Compute the fuzzy logic output
        self.fertilizer_sim.compute()

        # Get the fertilizer amounts needed
        n_fertilizer_needed = self.fertilizer_sim.output['n_fertilizer']
        p_fertilizer_needed = self.fertilizer_sim.output['p_fertilizer']
        k_fertilizer_needed = self.fertilizer_sim.output['k_fertilizer']

        return n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed

    def get_npk_needs(self, growth_stage):
        # Get the NPK needs based on the growth stage
        return self.growth_stages[growth_stage]['NPK']

    def calculate_fertilizer_amounts(self, n_fertilizer_needed, p_fertilizer_needed, k_fertilizer_needed):
        amounts = {}
        total_n_applied = total_p_applied = total_k_applied = 0
    
        # Count available fertilizers with N, P, and K to distribute evenly
        n_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["N"] > 0]
        p_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["P"] > 0]
        k_fertilizers = [fert for fert, nutr in self.fertilizers.items() if nutr["K"] > 0]
        
        # Distribute N evenly across all nitrogen-heavy and balanced fertilizers
        for fertilizer in n_fertilizers:
            n_content = self.fertilizers[fertilizer]["N"]
            n_share = n_fertilizer_needed / len(n_fertilizers)
            n_amount = n_share / n_content if n_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + n_amount
            total_n_applied += n_amount * n_content
    
        # Distribute P evenly across all phosphorus-heavy and balanced fertilizers
        for fertilizer in p_fertilizers:
            p_content = self.fertilizers[fertilizer]["P"]
            p_share = p_fertilizer_needed / len(p_fertilizers)
            p_amount = p_share / p_content if p_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + p_amount
            total_p_applied += p_amount * p_content
    
        # Distribute K evenly across all potassium-heavy and balanced fertilizers
        for fertilizer in k_fertilizers:
            k_content = self.fertilizers[fertilizer]["K"]
            k_share = k_fertilizer_needed / len(k_fertilizers)
            k_amount = k_share / k_content if k_content > 0 else 0
            amounts[fertilizer] = amounts.get(fertilizer, 0) + k_amount
            total_k_applied += k_amount * k_content
    
        return amounts

    def display_fertilizer_plan(self):
        fertilizer_plan_list = []

        print(f"\nFertilizer plan for {self.crop_name}:")
        for phase, stage_data in self.growth_stages.items():
            # Get fertilizer amounts using fuzzy logic
            n_fertilizer, p_fertilizer, k_fertilizer = self.fuzzy_logic(phase)
            fertilizer_amounts = self.calculate_fertilizer_amounts(n_fertilizer, p_fertilizer, k_fertilizer)

            # Display the data
            print(f"{phase} (Days: {stage_data['days'][0]} to {stage_data['days'][1]}):")
            print(f"  N fertilizer needed: {n_fertilizer:.2f} kg/ha")
            print(f"  P fertilizer needed: {p_fertilizer:.2f} kg/ha")
            print(f"  K fertilizer needed: {k_fertilizer:.2f} kg/ha")

            for fertilizer, amount in fertilizer_amounts.items():
                print(f"    {fertilizer}: {amount:.2f} kg/ha")

            print()

            # Store the data for this phase
            phase_data = {
                'phase': phase,
                'days': stage_data['days'],
                'n_fertilizer_needed': n_fertilizer,
                'p_fertilizer_needed': p_fertilizer,
                'k_fertilizer_needed': k_fertilizer,
                'specific_fertilizer_amounts': fertilizer_amounts
            }

            fertilizer_plan_list.append(phase_data)

        return fertilizer_plan_list

# Example usage
calculator = NPKComplexFertilizerCalculator_rice(
    crop_name="Rice", soil_n=30, soil_p=10, soil_k=15, biofertilizer=True
)
calculator.display_fertilizer_plan()


Fertilizer plan for Rice:
Seedling (Days: 0 to 21):
  N fertilizer needed: 60.00 kg/ha
  P fertilizer needed: 25.00 kg/ha
  K fertilizer needed: 30.00 kg/ha
    10:26:26: 148.08 kg/ha
    12:32:16: 145.83 kg/ha
    15:15:15: 150.00 kg/ha
    Urea (46-0-0): 21.74 kg/ha
    DAP (18-46-0): 66.43 kg/ha
    Ammonium Sulfate (21-0-0): 47.62 kg/ha
    SSP (0-16-0): 31.25 kg/ha
    MOP (0-0-60): 12.50 kg/ha

Tillering (Days: 22 to 45):
  N fertilizer needed: 60.00 kg/ha
  P fertilizer needed: 25.00 kg/ha
  K fertilizer needed: 30.00 kg/ha
    10:26:26: 148.08 kg/ha
    12:32:16: 145.83 kg/ha
    15:15:15: 150.00 kg/ha
    Urea (46-0-0): 21.74 kg/ha
    DAP (18-46-0): 66.43 kg/ha
    Ammonium Sulfate (21-0-0): 47.62 kg/ha
    SSP (0-16-0): 31.25 kg/ha
    MOP (0-0-60): 12.50 kg/ha

Panicle Initiation (Days: 46 to 65):
  N fertilizer needed: 45.07 kg/ha
  P fertilizer needed: 18.76 kg/ha
  K fertilizer needed: 21.18 kg/ha
    10:26:26: 109.92 kg/ha
    12:32:16: 107.42 kg/ha
    15:15:15: 110.3

[{'phase': 'Seedling',
  'days': (0, 21),
  'n_fertilizer_needed': 60.00000000000001,
  'p_fertilizer_needed': 25.00000000000001,
  'k_fertilizer_needed': 30.000000000000004,
  'specific_fertilizer_amounts': {'10:26:26': 148.0769230769231,
   '12:32:16': 145.83333333333337,
   '15:15:15': 150.00000000000003,
   'Urea (46-0-0)': 21.739130434782613,
   'DAP (18-46-0)': 66.42512077294687,
   'Ammonium Sulfate (21-0-0)': 47.61904761904763,
   'SSP (0-16-0)': 31.25000000000001,
   'MOP (0-0-60)': 12.500000000000002}},
 {'phase': 'Tillering',
  'days': (22, 45),
  'n_fertilizer_needed': 60.00000000000001,
  'p_fertilizer_needed': 25.00000000000001,
  'k_fertilizer_needed': 30.000000000000004,
  'specific_fertilizer_amounts': {'10:26:26': 148.0769230769231,
   '12:32:16': 145.83333333333337,
   '15:15:15': 150.00000000000003,
   'Urea (46-0-0)': 21.739130434782613,
   'DAP (18-46-0)': 66.42512077294687,
   'Ammonium Sulfate (21-0-0)': 47.61904761904763,
   'SSP (0-16-0)': 31.25000000000001,
 

In [1]:
import requests
from math import exp
from datetime import datetime, timedelta

# Constants
Cn = 900  # constant for the Penman-Monteith equation
Cd = 0.34  # constant for the Penman-Monteith equation
Kc = 1.15  # crop coefficient for mid-season maize
Eff = 0.85  # irrigation system efficiency
LOCATION = "Sangli"  # Location for weather data
API_KEY = "9e2f039c7cfb45498bb123017241910"  # Replace with your actual API key

# Fetch historical weather data for a specific date
def fetch_historical_weather_data(api_key, location, date):
    url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={location}&dt={date}"
    response = requests.get(url)
    return response.json()

# Calculate reference evapotranspiration (ET0) using the Penman-Monteith equation
def calculate_ET0(Tmax, Tmin, RHmean, u2, Rs):
    # Saturation vapor pressure (es)
    es = 0.6108 * (exp((17.27 * Tmax) / (Tmax + 237.3)) + exp((17.27 * Tmin) / (Tmin + 237.3))) / 2
    ea = es * RHmean / 100  # Actual vapor pressure

    # Net radiation (assuming simplified, based on solar radiation Rs)
    Rn = Rs * 0.408  # converting MJ/m² to mm/day equivalent

    # Mean temperature
    Tmean = (Tmax + Tmin) / 2

    # Slope of the vapor pressure curve (Δ)
    delta = 4098 * es / ((Tmean + 237.3) ** 2)

    # Psychrometric constant (γ)
    gamma = 0.665 * 101.3 / 1000  # Assuming sea level pressure in kPa

    # Penman-Monteith equation for reference evapotranspiration (ET0)
    ET0 = (0.408 * delta * Rn + gamma * Cn * u2 * (es - ea) / (Tmean + 273)) / (delta + gamma * (1 + Cd * u2))

    return ET0

# Calculate daily irrigation water requirement
def calculate_daily_irrigation_requirement(ET0, effective_precipitation):
    ETa = Kc * ET0  # Actual evapotranspiration (ETa)
    IWR = (ETa - effective_precipitation) / Eff  # Daily irrigation water requirement
    return IWR

# Convert mm to L/ha
def convert_mm_to_L_per_ha(mm):
    return mm * 10000  # 1 mm of water over 1 hectare is equivalent to 10,000 liters

# Main function to calculate irrigation needs over the past year
def calculate_irrigation_past_year(api_key, location):
    daily_irrigation_requirements = []

    # Define the date range: past year
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)

    # Loop through each date in the range
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        weather_data = fetch_historical_weather_data(api_key, location, date_str)

        # Extract relevant data from the historical weather API response
        day_data = weather_data['forecast']['forecastday'][0]  # Only one day of data
        Tmax = day_data['day']['maxtemp_c']  # Max temperature in Celsius
        Tmin = day_data['day']['mintemp_c']   # Min temperature in Celsius
        RHmean = day_data['day']['avghumidity']  # Average humidity
        u2 = day_data['day']['maxwind_kph'] * 1000 / 3600  # Convert wind speed to m/s
        Rs = day_data['day']['uv']  # Assuming UV index as a proxy for solar radiation
        effective_precipitation = day_data['day']['totalprecip_mm']  # Total precipitation in mm

        # Calculate ET0
        ET0 = calculate_ET0(Tmax, Tmin, RHmean, u2, Rs)

        # Calculate daily irrigation water requirement
        daily_irrigation = calculate_daily_irrigation_requirement(ET0, effective_precipitation)
        daily_irrigation_l_per_ha = convert_mm_to_L_per_ha(daily_irrigation)

        # Store the result
        if daily_irrigation < 0:
            daily_irrigation_requirements.append(f"{date_str}: No irrigation needed (Excess moisture)")
        else:
            daily_irrigation_requirements.append(f"{date_str}: {daily_irrigation_l_per_ha:.2f} L/ha")

        # Move to the next day
        current_date += timedelta(days=1)

    return daily_irrigation_requirements

# Execute the main calculation
if __name__ == "__main__":
    irrigation_requirements = calculate_irrigation_past_year(API_KEY, LOCATION)

    print("Daily irrigation water requirements (L/ha) for each day in the past year:")
    for requirement in irrigation_requirements:
        print(requirement)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests
import json

def get_weather(api_key, city):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,  # Corrected the API key variable
        "units": "metric"  # You can change this to "imperial" if you prefer Fahrenheit
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if response.status_code == 200:
        return data
    else:
        print("Error:", data['message'])
        return None

def main():
    api_key = "ca394e8a960bf09af5cc0b1970083cf4"  # Replace with your actual API key
    city = input("Enter city name: ")

    weather_data = get_weather(api_key, city)

    if weather_data:
        temperature = weather_data['main']['temp']
        description = weather_data['weather'][0]['description']

        print(f"Weather in {city}:")
        print(f"Temperature: {temperature}°C")
        print(f"Description: {description.capitalize()}")

if __name__ == "__main__":
    main()

In [1]:
import requests
from math import exp
from datetime import datetime, timedelta
import asyncio
import aiohttp
from typing import List, Dict
import pandas as pd
import numpy as np
import nest_asyncio

# Enable nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

# Constants remain the same
Cn = 900
Cd = 0.34
Kc = 1.15
Eff = 0.85
LOCATION = "Sangli"
API_KEY = "9e2f039c7cfb45498bb123017241910"

async def fetch_weather_data_batch(session: aiohttp.ClientSession, location: str, dates: List[str]) -> List[Dict]:
    """Fetch weather data for multiple dates in parallel"""
    async def fetch_single_date(date: str):
        url = f"http://api.weatherapi.com/v1/history.json?key={API_KEY}&q={location}&dt={date}"
        async with session.get(url) as response:
            return await response.json()
    
    # Create tasks for all dates
    tasks = [fetch_single_date(date) for date in dates]
    return await asyncio.gather(*tasks)

def calculate_ET0(Tmax, Tmin, RHmean, u2, Rs):
    # Calculation remains the same but vectorized for pandas
    es = 0.6108 * (np.exp((17.27 * Tmax) / (Tmax + 237.3)) + np.exp((17.27 * Tmin) / (Tmin + 237.3))) / 2
    ea = es * RHmean / 100
    Rn = Rs * 0.408
    Tmean = (Tmax + Tmin) / 2
    delta = 4098 * es / ((Tmean + 237.3) ** 2)
    gamma = 0.665 * 101.3 / 1000
    ET0 = (0.408 * delta * Rn + gamma * Cn * u2 * (es - ea) / (Tmean + 273)) / (delta + gamma * (1 + Cd * u2))
    return ET0

def calculate_daily_irrigation_requirement(ET0, effective_precipitation):
    ETa = Kc * ET0
    return (ETa - effective_precipitation) / Eff

async def calculate_irrigation_past_year(location: str):
    # Generate all dates
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    dates = [(start_date + timedelta(days=x)).strftime('%Y-%m-%d') 
             for x in range((end_date - start_date).days + 1)]
    
    # Create batches of dates (to avoid overwhelming the API)
    BATCH_SIZE = 20
    date_batches = [dates[i:i + BATCH_SIZE] for i in range(0, len(dates), BATCH_SIZE)]
    
    all_weather_data = []
    async with aiohttp.ClientSession() as session:
        for batch in date_batches:
            batch_data = await fetch_weather_data_batch(session, location, batch)
            all_weather_data.extend(batch_data)
    
    # Process all data at once using pandas
    records = []
    for date, weather in zip(dates, all_weather_data):
        day_data = weather['forecast']['forecastday'][0]
        records.append({
            'date': date,
            'Tmax': day_data['day']['maxtemp_c'],
            'Tmin': day_data['day']['mintemp_c'],
            'RHmean': day_data['day']['avghumidity'],
            'u2': day_data['day']['maxwind_kph'] * 1000 / 3600,
            'Rs': day_data['day']['uv'],
            'precipitation': day_data['day']['totalprecip_mm']
        })
    
    # Convert to DataFrame for vectorized operations
    df = pd.DataFrame(records)
    
    # Calculate irrigation requirements in vectorized form
    df['ET0'] = calculate_ET0(
        df['Tmax'], df['Tmin'], df['RHmean'], df['u2'], df['Rs']
    )
    df['irrigation_mm'] = calculate_daily_irrigation_requirement(
        df['ET0'], df['precipitation']
    )
    df['irrigation_L_per_ha'] = df['irrigation_mm'] * 10000
    
    # Format results
    results = []
    for _, row in df.iterrows():
        if row['irrigation_mm'] < 0:
            results.append(f"{row['date']}: No irrigation needed (Excess moisture)")
        else:
            results.append(f"{row['date']}: {row['irrigation_L_per_ha']:.2f} L/ha")
    
    return results

def run_irrigation_calculation(location: str):
    """Wrapper function to handle async execution"""
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    return loop.run_until_complete(calculate_irrigation_past_year(location))

if __name__ == "__main__":
    # Install required packages if not already installed
    try:
        import nest_asyncio
    except ImportError:
        print("Installing required packages...")
        import subprocess
        subprocess.check_call(['pip', 'install', 'nest_asyncio', 'aiohttp', 'pandas', 'numpy'])
        import nest_asyncio
    
    # Run the calculation
    results = run_irrigation_calculation(LOCATION)
    
    print("Daily irrigation water requirements (L/ha) for each day in the past year:")
    for result in results:
        print(result)

Daily irrigation water requirements (L/ha) for each day in the past year:
2023-10-26: 41881.12 L/ha
2023-10-27: 40569.65 L/ha
2023-10-28: 45507.05 L/ha
2023-10-29: 41858.00 L/ha
2023-10-30: 58230.23 L/ha
2023-10-31: 24946.99 L/ha
2023-11-01: 42303.51 L/ha
2023-11-02: 54537.06 L/ha
2023-11-03: 36821.59 L/ha
2023-11-04: 46939.51 L/ha
2023-11-05: 40121.49 L/ha
2023-11-06: 36615.89 L/ha
2023-11-07: 45448.16 L/ha
2023-11-08: 12925.62 L/ha
2023-11-09: 24189.76 L/ha
2023-11-10: 36472.04 L/ha
2023-11-11: 18023.47 L/ha
2023-11-12: 24218.88 L/ha
2023-11-13: 34158.23 L/ha
2023-11-14: 45969.13 L/ha
2023-11-15: 39579.44 L/ha
2023-11-16: 75865.05 L/ha
2023-11-17: 64704.67 L/ha
2023-11-18: 18586.64 L/ha
2023-11-19: 35166.04 L/ha
2023-11-20: 63929.82 L/ha
2023-11-21: 40314.84 L/ha
2023-11-22: 13798.87 L/ha
2023-11-23: 37972.23 L/ha
2023-11-24: 39649.03 L/ha
2023-11-25: 45779.03 L/ha
2023-11-26: 54329.16 L/ha
2023-11-27: 55783.36 L/ha
2023-11-28: 21959.68 L/ha
2023-11-29: 11012.94 L/ha
2023-11-30: 2182